In [44]:
import requests
from tqdm import tqdm

## Hometask #1

### Maksim Sorokin <maxt.coder@gmail.com>

In [45]:
def getTotal():
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=1&limit=100&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    return res.json().get('data').get('total')

In [46]:
print('Всего объектов: '+ str(getTotal()))

Всего объектов: 10616


In [47]:
def getIds(offset_ = 1, limit_ = 100):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_data.get('data').get('list')[0]
    objects_list = objects_data.get('data').get('list')
    l = list()
    for a in objects_list:
        l.append(a['objId'])
    return l

In [48]:
all_objects_list = list()
work_in_progress = True
offset = 1
total_objects =  getTotal()
iterations = total_objects // 100
print('Expected iterations count: ')
if (total_objects % 100) > 0:
    iterations+=1

for a in tqdm(range(iterations)):
    all_objects_list.extend(getIds(a*100))

assert len(all_objects_list) == total_objects, 'Didn\'t load the whole dataset'
print('Loaded '+str(len(all_objects_list)) + ' entries, total '+str(total_objects))


Expected iterations count: 


100%|██████████| 107/107 [02:33<00:00,  1.43s/it]

Loaded 10616 entries, total 10616


In [50]:
    def get_object_by_id(id):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    answ = requests.get(url)
    return answ.json()

In [52]:
all_objects_full_list = list()
ids_with_errors = list()
for a in tqdm(all_objects_list):
    try:
        all_objects_full_list.append(get_object_by_id(a))
    except:
        ids_with_errors.append(a)

print('Additional processing of failed ids')
for b in tqdm(ids_with_errors):
    all_objects_full_list.append(get_object_by_id(b))

assert len(all_objects_full_list) == total_objects, 'Didn\'t load the whole dataset'

100%|██████████| 10616/10616 [26:10<00:00,  6.76it/s] 


Additional processing of failed ids


100%|██████████| 1/1 [00:00<00:00,  8.09it/s]


In [54]:
len(all_objects_full_list)

10616

In [76]:
import pandas
result_df = pandas.DataFrame()
for c in tqdm(all_objects_full_list):
    temp_result_dict = dict(c.get('data'))
    developer_dict = temp_result_dict.get('developer')
    if developer_dict:
        devId = developer_dict.get('devId')
        del temp_result_dict['developer']
        temp_result_dict['devId'] = devId
    temp_df = pandas.DataFrame([temp_result_dict])
    result_df = result_df = pandas.concat([result_df,temp_df], ignore_index = True)
result_df

100%|██████████| 10616/10616 [02:37<00:00, 67.34it/s]


,id,pdId,region,address,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objElemParkingCnt,...,generalContractorNm,nonlivFirstFloor,objLkSaleDepEmail,transportDistIndex,transportDistIndexValue,nameObj,greenAreaIndexValue,airQualityIndexValue,complexShortNm,metro
0,29679,124647,61,г Таганрог,18,2020-03-31,Другое,Под ключ,Есть,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13391,28101,72,"г Тюмень, ул Профсоюзная",44,2022-12-31,Другое,По желанию заказчика,Есть,41,...,ООО Транспромжилстрой-2005 (ИНН: 7204097730),1.0,info@2mengroup.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13399,28141,72,"г Тюмень, ул Профсоюзная, д. 56",0,2024-12-31,Другое,Без отделки,Нет,125,...,ООО Транспромжилстрой-2005 (ИНН: 7204097730),1.0,info@2mengroup.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13383,28105,72,г Тюмень,68,2022-12-31,Другое,По желанию заказчика,Есть,31,...,ООО Транспромжилстрой-2005 (ИНН: 7204097730),1.0,info@2mengroup.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13398,28141,72,"г Тюмень, ул Профсоюзная, д. 56",43,2024-12-31,Другое,Без отделки,Нет,0,...,ООО Транспромжилстрой-2005 (ИНН: 7204097730),0.0,info@2mengroup.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10611,28072,96379,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",312,2024-06-30,Другое,Под ключ,Нет,0,...,NaN,1.0,info@bw-estate.ru,NaN,NaN,"Жилой комплекс""Томилинский бульвар""",NaN,7.0,NaN,NaN
10612,28070,99487,50,"г Котельники, район мкр.Ковровый, д.1-5, 24-29...",768,2024-06-30,Другое,Под ключ,Нет,0,...,NaN,1.0,info@bw-estate.ru,NaN,NaN,"Жилой комплекс""Томилинский бульвар""",NaN,7.0,NaN,NaN
10613,44447,475093,76,"г Ярославль, Фрунзенский район, территория в р...",127,2022-12-31,Кирпич,Другое,Нет,0,...,"АО Специализированный застройщик ""Горстройзака...",0.0,yarzak@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10614,45321,516648,32,"г Брянск, улица Молодой Гвардии, строение Поз.1",90,2023-03-31,Монолит-кирпич,Без отделки,Нет,0,...,NaN,0.0,saks_br@mail.ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
import openpyxl
result_df.to_excel('output/dom_objects.xlsx')

In [79]:
result_df.to_pickle("output/dom_objeckts.pkl")

In [84]:
from sqlalchemy import create_engine
result_df_str =  result_df.applymap(str)
engine = create_engine('sqlite://', echo=False)
result_df_str.to_sql('dom_objects', con=engine)
for row in engine.execute("select count(*) from dom_objects"):
    print(row)

(10616,)
